In [ ]:
from autumn.infrastructure.remote import springboard
from autumn.projects.sm_covid2.common_school.runner_tools import run_full_analysis, print_continuous_status, download_analysis

#### Standard config

In [ ]:
ISO3_LIST = ["AUS", "BOL", "FRA", "MAR", "PHL"]
ANALYSIS = "main"

N_CHAINS = 32
N_OPTI_SEARCHES = 32
N_BEST_RETAINED = 16
OPTI_BUDGET = 10000

METROPOLIS_TUNE = 2000
METROPOLIS_DRAWS = 10000
METROPOLIS_METHOD = "DEMetropolis"

FULL_RUNS_SAMPLES = 1000
BURN_IN = 5000

In [ ]:
def remote_full_analysis_task(bridge: springboard.task.TaskBridge, iso3: str = 'FRA', analysis: str = "main"):
    
    import multiprocessing as mp
    mp.set_start_method('forkserver')

    bridge.logger.info(f"Running full analysis for {iso3}. Analysis type: {analysis}.")

    idata, uncertainty_df, diff_quantiles_df = run_full_analysis(
        iso3,
        analysis=analysis, 
        opti_params={'n_searches': N_OPTI_SEARCHES, "n_best_retained": N_BEST_RETAINED, 'num_workers': 8, 'parallel_opti_jobs': 8, 'warmup_iterations': 0, 'search_iterations': OPTI_BUDGET, 'init_method': 'LHS'},
        mcmc_params={'draws': METROPOLIS_DRAWS, 'tune': METROPOLIS_TUNE, 'cores': N_CHAINS, 'chains': N_CHAINS, 'method': METROPOLIS_METHOD},
        full_run_params={'samples': FULL_RUNS_SAMPLES, 'burn_in': BURN_IN},
        output_folder=bridge.out_path,
        logger=bridge.logger
    )
   
    bridge.logger.info("Full analysis complete")

In [ ]:
mspec = springboard.EC2MachineSpec(N_CHAINS, 4, "compute")

In [ ]:
analysis_title = "test_run"
config_str = f"_{ANALYSIS}_LHS{N_OPTI_SEARCHES}_opt{OPTI_BUDGET}_mc{METROPOLIS_TUNE}n{METROPOLIS_DRAWS}"
run_paths = [springboard.launch.get_autumn_project_run_path("school_project", iso3, analysis_title + config_str) for iso3 in ISO3_LIST]
run_paths

In [ ]:
task_dict = {}
for i, iso3 in enumerate(ISO3_LIST):
    # Use these targets, but keep the other kwargs the same
    task_kwargs = {
        "iso3": iso3,
        "analysis": ANALYSIS
    }
    task_dict[run_paths[i]] = springboard.TaskSpec(remote_full_analysis_task, task_kwargs)

In [ ]:
runners = springboard.launch.launch_synced_multiple_autumn_task(task_dict, mspec)

In [ ]:
runners_dict = {iso3: runners[list(runners.keys())[i]] for i, iso3 in enumerate(ISO3_LIST)}

In [ ]:
runner.instance

In [ ]:
runners_dict["AUS"].s3.get_status()

In [ ]:
for iso3 in ISO3_LIST:
    runner = runners_dict[iso3]
    print(runner.s3.get_status())
    print(runner.top("%MEM"))

In [ ]:
run_paths

In [ ]:
for run_path in run_paths:
    download_analysis(run_path)